In [1]:
param_make = 'ACURA'

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./' + param_make + '/4.engine.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_make = df['Make'].tolist()
list_year = df['Year'].tolist()
list_model = df['Model'].tolist()
list_engine = df['Engine'].tolist()
list_car_code = df['Car_Code'].tolist()

print('总数量：' + str(len(list_engine)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from urllib.parse import quote
import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for make, year, model, engine, car_code in zip(list_make, list_year, list_model, list_engine, list_car_code):
    work.put_nowait((make, year, model, engine, car_code))

def crawler():
    global df
    
    while not work.empty():
        make, year, model, engine, car_code = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(31):
            try:
                data = 'func=navnode_fetch&payload=' + quote('{"jsn":{"tab":"catalog","make":"') + make.replace(' ', '+') + quote('","year":"') + year + quote('","model":"') + model.replace('+', '%2B').replace(' ', '+').replace('&', '%26').replace('/', '%2F') + quote('","engine":"') + engine.replace(' ', '+').replace('/', '%2F') + quote('","carcode":"') + car_code + quote('","nodetype":"carcode"}}') + '&api_json_request=1&sctchecked=1&scbeenloaded=true&curCartGroupID='

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers={'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'},
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['html_fill_sections']['navchildren[]']
                
                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_category = html.xpath('//td[@class="nlabel"]/a/text()')

                # = = = = = = = = = = = = = = =
                
                list_url = html.xpath('//td[@class="niconspace ncollapsedicon"]/a/@href')

                list_category_code = [_.split(',')[-1] for _ in list_url]

                # = = = = = = = = = = = = = = =

                list_url = ['https://www.rockauto.com'+_ for _ in list_url]

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'status': 'ok',
                                        'Make': make,
                                        'Year': year,
                                        'Model': model,
                                        'Engine': engine,
                                        'Car_Code': car_code,
                                        'Category': list_category,
                                        'Category_Code': list_category_code,
                                        'Url': list_url})

                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Make': make,
                                     'Year': year,
                                     'Model': model,
                                     'Engine': engine,
                                     'Car_Code': car_code}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(make + ', ' + year + ', ' + model + ', ' + engine +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./' + param_make + '/category_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：199

ACURA, 2022, MDX, 3.5L V6  <->  [ok] - 剩余数量：179
ACURA, 2024, RDX, 2.0L L4 Turbocharged  <->  [ok] - 剩余数量：178
ACURA, 2024, MDX, 3.0L V6 Turbocharged  <->  [ok] - 剩余数量：177
ACURA, 2022, RDX, 2.0L L4 Turbocharged  <->  [ok] - 剩余数量：176
ACURA, 2024, INTEGRA, 1.5L L4 Turbocharged  <->  [ok] - 剩余数量：175
ACURA, 2023, INTEGRA, 1.5L L4 Turbocharged  <->  [ok] - 剩余数量：174
ACURA, 2022, MDX, 3.0L V6 Turbocharged  <->  [ok] - 剩余数量：173
ACURA, 2023, TLX, 3.0L V6 Turbocharged  <->  [ok] - 剩余数量：172
ACURA, 2023, MDX, 3.0L V6 Turbocharged  <->  [ok] - 剩余数量：171
ACURA, 2023, RDX, 2.0L L4 Turbocharged  <->  [ok] - 剩余数量：170
ACURA, 2022, NSX, 3.5L V6 ELECTRIC/GAS Turbocharged  <->  [ok] - 剩余数量：169
ACURA, 2022, ILX, 2.4L L4  <->  [ok] - 剩余数量：168
ACURA, 2021, TLX, 3.0L V6 Turbocharged  <->  [ok] - 剩余数量：167
ACURA, 2021, NSX, 3.5L V6 ELECTRIC/GAS Turbocharged  <->  [ok] - 剩余数量：166
ACURA, 2021, TLX, 2.0L L4 Turbocharged  <->  [ok] - 剩余数量：165
ACURA, 2020, MDX, 3.5L V6  <->  [ok] - 剩余数量：164
ACURA, 2023, MDX, 3.

ACURA, 2005, RSX, 2.0L L4  <->  [ok] - 剩余数量：17
ACURA, 2002, CL, 3.2L V6  <->  [ok] - 剩余数量：16
ACURA, 2005, NSX, 3.0L V6  <->  [ok] - 剩余数量：15
ACURA, 2002, EL, 1.7L L4  <->  [ok] - 剩余数量：14
ACURA, 2003, CL, 3.2L V6  <->  [ok] - 剩余数量：13
ACURA, 2002, MDX, 3.5L V6  <->  [ok] - 剩余数量：12
ACURA, 2003, EL, 1.7L L4  <->  [ok] - 剩余数量：11
ACURA, 2003, RL, 3.5L V6  <->  [ok] - 剩余数量：10
ACURA, 2004, TL, 3.2L V6  <->  [ok] - 剩余数量：9
ACURA, 2001, CL, 3.2L V6  <->  [ok] - 剩余数量：8
ACURA, 2002, RSX, 2.0L L4  <->  [ok] - 剩余数量：7
ACURA, 2005, NSX, 3.2L V6  <->  [ok] - 剩余数量：6
ACURA, 2001, NSX, 3.2L V6  <->  [ok] - 剩余数量：5
ACURA, 2001, MDX, 3.5L V6  <->  [ok] - 剩余数量：4
ACURA, 2004, NSX, 3.0L V6  <->  [ok] - 剩余数量：3
ACURA, 2001, RL, 3.5L V6  <->  [ok] - 剩余数量：2
ACURA, 2002, NSX, 3.0L V6  <->  [ok] - 剩余数量：1
ACURA, 2014, TL, 3.7L V6  <->  [ok] - 剩余数量：0
ACURA, 2000, NSX, 3.0L V6  <->  [ok] - 剩余数量：0
ACURA, 2004, MDX, 3.5L V6  <->  [ok] - 剩余数量：0
ACURA, 2001, EL, 1.7L L4  <->  [ok] - 剩余数量：0
ACURA, 2000, TL, 3.2L V6  <->  [ok] 